In [1]:
import pandas as pd
from backtesting import Backtest, Strategy


/opt/homebrew/Caskroom/miniconda/base/envs/bot/lib/python3.13/site-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

# Reading in data and setting up training and testing sets
- Here we have set up a 70 / 30 training / testing set data split
- The reason for setting up training and testing sets is so we can optimise parameters on the training set, and then test them on our testing set to see if the parameters work on unseen data

In [2]:
df = pd.read_csv('sol_1h_data.csv', parse_dates=True, index_col=0)
print(df)


split_date = df.index[int(len(df) * 0.7)]

#split data into training and testing sets
df_train = df.loc[:split_date]
df_test = df.loc[split_date:]

print('Training set:')
print(df_train.describe())
print('Testing set:')
print(df_test.describe())





                       Open    High     Low   Close
timestamp                                          
2023-10-30 13:00:00   34.76   35.05   34.60   34.70
2023-10-30 14:00:00   34.70   35.00   34.51   34.80
2023-10-30 15:00:00   34.81   35.13   34.78   35.04
2023-10-30 16:00:00   35.04   35.05   34.78   34.88
2023-10-30 17:00:00   34.88   34.96   34.23   34.89
...                     ...     ...     ...     ...
2025-10-29 08:00:00  195.65  195.73  194.02  194.50
2025-10-29 09:00:00  194.50  194.74  193.78  194.21
2025-10-29 10:00:00  194.20  197.07  193.95  196.25
2025-10-29 11:00:00  196.26  199.37  196.04  198.74
2025-10-29 12:00:00  198.74  199.60  198.15  199.46

[17520 rows x 4 columns]
Training set:
               Open          High           Low         Close
count  12265.000000  12265.000000  12265.000000  12265.000000
mean     149.126347    150.215795    147.995587    149.135415
std       48.727494     49.060668     48.384326     48.716787
min       34.700000     34.890000   

# Defining our strategy and its parameters
- We define our strategy as `SMAStrategy` which is an instance of `Strategy` (a built in class)
- We define the parameters that our strategy will use
- Our strategy has to have these two functions: `init` and `next`
## `init` function
- init (initialise) is the function ran when the strategy is created
- we use this function to define the logic that our trading bot will use
- in this example we precompute short and long simple moving averages (SMAs) of the Close price
- these precomputed values can then easily be used in the next function

## `next` function
- during the backtest, the next function will be ran on every single data point
- this is where we include the actual trade execution logic
- here we will enter a position (buy) when the short SMA crosses above the long SMA
- we will close the position (sell) when the short SMA crosses below the long SMA
- NOTE: this is simple logic where we only buy and sell spot, no shorts, we always buy with maximum available capital

In [ ]:
class SMAStrategy(Strategy):
    #define parameters
    sma_long_period = 30
    sma_short_period = 10

    def init(self):
        #calculate indicators
        close_prices = pd.Series(self.data.Close, index=self.data.index)
        sma_short = close_prices.rolling(window=self.sma_short_period).mean()
        sma_long = close_prices.rolling(window=self.sma_long_period).mean()
        self.sma_short = self.I(lambda: sma_short, name='SMA_SHORT')
        self.sma_long = self.I(lambda: sma_long, name='SMA_LONG')

    def next(self):
        #execute logic
        if self.sma_short[-1] > self.sma_long[-1] and self.sma_short[-2] <= self.sma_long[-2]:
            self.buy()
        elif self.sma_short[-1] < self.sma_long[-1] and self.sma_short[-2] >= self.sma_long[-2]:
            self.position.close()

# Running a backtest on the strategy
## Backtest function parameter explanation:
- `df_train` - our dataset that we will run the strategy on
- `SMAStrategy` - the instance of Strategy that we defined, the trading logic to be backtested
- `cash` - how much money the backtest will begin with
- `exclusive_orders` - whether or not we can have multiple trades open at the same time, in this instance True means we cannot
- `finalize_trades` - at the end of the backtest any open trades will be closed at the final price in the dataset

## Displaying key performance metrics
- here we print the stats from the backtest
- the power of backtesting.py is these metrics, they are all calculated for us allowing us to easily evaluate a strategy across these metrics

In [ ]:
#run the Backtest function
bt_train = Backtest(...)
stats = ...
print(stats)

# Plotting the strategy to see a visualisation of the performance

In [ ]:
#plot the strategy 

# Now to optimise the strategy
- Our strategy has parameters (e.g. `sma_short_period`, `sma_long_period`) we can optimise the strategy to find the best parameters
- We can optimise for whatever we would like, in this instance we optimise for Sharpe Ratio
- The optimise function will simply run through all possible combinations of parameters and find the one with the highest Sharpe Ratio
- This can be slow, so we can add constraints for parameter combinations that we will allow. e.g. `sma_short_period` must be lower than `sma_long_period`


In [ ]:
def optimise(series):
    #create an optimisation function to maximise for
    pass

def constraint(series):
    #set up constraints to limit search space
    pass


# Running the optimisation
- We provide ranges of values for each parameter that we want to be tested
- `sma_short_period=range(10,20,5)` will test all combinations of the short SMA period from 10 to 20 incrementing by 5 each time. That is 10, 15, 20
- as it is doing an exhaustive search, running backtests for all combinations, it can take a long time if we specify lots of values
- take note that we specified to maximize for our optimise function (`maximize=optimise`), so it will find the parameters that maximise for whatever our optimise function returns. 
- It is important to note that if we wanted to optimise to minimise a metric we would change this, e.g. minimise the maximum drawdown faced by the strategy


In [ ]:
stats_optimised = bt_train.optimize(
    #set ranges of parameters to test
    sma_short_period=..., 
    sma_long_period=..., 
    maximize=optimise, 
    constraint=constraint)

In [ ]:
print(stats)
stats._strategy

# Now we will test the optimized parameters on the testing set
- Here, we pass in `df_test`, our testing dataset which contains unseen data.  
- We again call `.run()`, but this time we pass in our optimized parameters `.run(**stats_optimised._strategy._params)`. This runs the backtest using those parameters rather than the ones defined in the Strategy class.  
- Finally, we print the performance statistics of this backtest to see how our optimized parameters performed on the testing set.  

## Did we overfit?
- We want to determine whether the optimized parameters were overfitted to the training set.  
- Overfitting occurs when a strategy learns patterns that are too specific to the training data, failing to capture the general, repeatable signals we are aiming for.  
- As a result, it performs well on the training data but fails to replicate that performance on unseen testing data.
- We can compare performance on training set and testing set to determine if the parameters were overfit


In [ ]:
bt_test = Backtest()
#run the backtest with optimised parameters
test_stats = bt_test.run(...)

#printing all stats side by side for comparison
print(f"{'Metric':<30} | {'TRAINING (Before)':<25} | {'TRAINING (Optimized)':<25} | {'TEST SET':<25}")
print("-" * 115)

for metric in stats.index:
    if metric not in ['_strategy', '_equity_curve', '_trades']:
        train_before_val = stats[metric]
        train_after_val = stats_optimised[metric]
        test_val = test_stats[metric]
        print(f"{metric:<30} | {str(train_before_val):<25} | {str(train_after_val):<25} | {str(test_val):<25}")


# Plotting the optimised strategy on our test set to visualise its performance


In [ ]:
#plot the optimised strategy
